In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import math
import sklearn
from scipy import stats
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn import linear_model
import patsy
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

1.a.

In [59]:
data1 = pd.read_excel("RawDataFinal1.xlsx")
train1, test1 = train_test_split(data1, test_size=0.2,random_state=3)

In [44]:
#results from the baseline model
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train1, test_size=0.2)
    X = training.drop(columns=['SalePrice'])
    Y = training['SalePrice']
    test_x=testing.drop(columns=['SalePrice'])
    test_y=testing['SalePrice']
    lm = smf.ols('SalePrice ~ '+' + '.join(list(X)), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

Estimated out of sample Root mean squared error for this model is  29679.1051548
Estimated out of sample R-square for this model is 0.851209179287
Estimated in sample R-square for this model is 0.88465163387


1.c.

In [45]:
data2 = pd.read_excel("NewData.xlsx")
singlefeatures = data2.columns[:-1]
features = list(singlefeatures)
for i in range(len(features)):
    features[i] = "".join(features[i].split())
    features[i] = features[i].replace('.','DOT')
singlefeatures = features
features.append('SalePrice')
data2.columns = features
train2, test2 = train_test_split(data2, test_size=0.2,random_state=3)


In [46]:
#results from the selected variable model
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train2, test_size=0.5)
    X = training.drop(columns=['SalePrice'])
    Y = training['SalePrice']
    test_x=testing.drop(columns=['SalePrice'])
    test_y=testing['SalePrice']
    lm = smf.ols('SalePrice ~ '+' + '.join(list(X)), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

Estimated out of sample Root mean squared error for this model is  34977.3884266
Estimated out of sample R-square for this model is 0.812686369025
Estimated in sample R-square for this model is 0.861790422022


Baseline model performs slightly better.

2.Building your model

In [47]:
#Linear regression with Lasso
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train1, test_size=0.2)
    X = training.drop(columns=['SalePrice'])
    Y = training['SalePrice']
    test_x=testing.drop(columns=['SalePrice'])
    test_y=testing['SalePrice']
    alpha=np.arange(0,30,0.1)
    singlefeatures = list(X)
    lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
    lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]
    lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

/Applications/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Applications/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:473: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)


Estimated out of sample Root mean squared error for this model is  37260.1485012
Estimated out of sample R-square for this model is 0.790966421847
Estimated in sample R-square for this model is 0.893823855579


In [54]:
#Linear regression with Ridge
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train1, test_size=0.2)
    X = training.drop(columns=['SalePrice'])
    Y = training['SalePrice']
    test_x=testing.drop(columns=['SalePrice'])
    test_y=testing['SalePrice']
    alpha=np.arange(0,30,0.1)
    ridge = linear_model.RidgeCV(alphas=alpha,cv =5).fit(X,Y)
    R2=ridge.score(X,Y)
    insample_r2.append(R2)
    Prediction = ridge.predict(test_x)
    R2_out = 1. - np.sum((Prediction-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(R2_out)
    rsme = sqrt(mean_squared_error(test_y, Prediction))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

Estimated out of sample Root mean squared error for this model is  33705.8380992
Estimated out of sample R-square for this model is 0.826904215575
Estimated in sample R-square for this model is 0.883152470449


In [55]:
#Linear regression with Lasso and log transformation
insample_r2=[]
outsample_r2=[]
outsample_rsme=[]
for i in range (5):
    training, testing = train_test_split(train1, test_size=0.2)
    training = training.set_index([list(range(len(training)))])
    for i in range(len(training["SalePrice"])):
        training.loc[i,('SalePrice')] = math.log(training.loc[i,('SalePrice')])    
    X = training.drop(columns=['SalePrice'])
    Y = training['SalePrice']
    testing = testing.set_index([list(range(len(testing)))])
    for i in range(len(testing["SalePrice"])):
        testing.loc[i,('SalePrice')] = math.log(testing.loc[i,('SalePrice')])
    test_x=testing.drop(columns=['SalePrice'])
    test_y=testing['SalePrice']
    alpha=np.arange(0,30,0.1)
    singlefeatures = list(X)
    lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
    lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-8]
    lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=training).fit()
    insample_r2.append(lm.rsquared)
    rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
    test_y.mean()-test_y)**2)
    outsample_r2.append(rsq)
    for i in range(len(testing["SalePrice"])):
        testing.loc[i,('SalePrice')] = math.exp(testing.loc[i,('SalePrice')])
    prediction = lm.predict(test_x)
    for i in range(len(prediction)):
        prediction[i]=math.exp(prediction[i])
    rsme = sqrt(mean_squared_error(test_y, prediction))
    outsample_rsme.append(rsme)
print "Estimated out of sample Root mean squared error for this model is ", sum(outsample_rsme)/5
print 'Estimated out of sample R-square for this model is',sum(outsample_r2)/5
print 'Estimated in sample R-square for this model is',sum(insample_r2)/5

Estimated out of sample Root mean squared error for this model is  31351.3111695
Estimated out of sample R-square for this model is 0.881967883428
Estimated in sample R-square for this model is 0.911742111157


3. Prediction on the test set

In [61]:
#Linear regression with Lasso and log transformation
outsample_r2=[]
outsample_rsme=[]
train1 = train1.set_index([list(range(len(train1)))])
for i in range(len(train1["SalePrice"])):
    train1.loc[i,('SalePrice')] = math.log(train1.loc[i,('SalePrice')])    
X = train1.drop(columns=['SalePrice'])
Y = train1['SalePrice']
test1 = test1.set_index([list(range(len(test1)))])
for i in range(len(test1["SalePrice"])):
    test1.loc[i,('SalePrice')] = math.log(test1.loc[i,('SalePrice')])
test_x=test1.drop(columns=['SalePrice'])
test_y=test1['SalePrice']
alpha=np.arange(0,30,0.1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-8]
lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=train1).fit()
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
test_y.mean()-test_y)**2)
outsample_r2.append(rsq)
for i in range(len(test1["SalePrice"])):
    test1.loc[i,('SalePrice')] = math.exp(test1.loc[i,('SalePrice')])
prediction = lm.predict(test_x)
for i in range(len(prediction)):
    prediction[i]=math.exp(prediction[i])
rsme = sqrt(mean_squared_error(test_y, prediction))
outsample_rsme.append(rsme)
print "Root mean squared error for this model is ", sum(outsample_rsme)/1
print 'R-square for this model is',sum(outsample_r2)/1


Root mean squared error for this model is  22191.8965145
R-square for this model is 0.906074169505


In [63]:
data1 = pd.read_excel("RawDataFinal1.xlsx")
train1, test1 = train_test_split(data1, test_size=0.2,random_state=3)

In [64]:
#Baseline model
outsample_r2=[]
outsample_rsme=[]

train1, test1 = train_test_split(train1, test_size=0.2)
X = train1.drop(columns=['SalePrice'])
Y = train1['SalePrice']
test_x=test1.drop(columns=['SalePrice'])
test_y=test1['SalePrice']
lm = smf.ols('SalePrice ~ '+' + '.join(list(X)), data=train1).fit()
rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
test_y.mean()-test_y)**2)
outsample_r2.append(rsq)
rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
outsample_rsme.append(rsme)
print "Root mean squared error for this model is ", sum(outsample_rsme)/1
print 'R-square for this model is',sum(outsample_r2)/1

Root mean squared error for this model is  26954.5168603
R-square for this model is 0.868857088199
